In [4]:
import os
import wfdb
import pandas as pd
import numpy as np
from tqdm import tqdm
import neurokit2 as nk
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [5]:
base_path = "/kaggle/input/ptb-xl-dataset/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/records500"

record_paths = []
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith(".dat"):
            record_paths.append(os.path.join(root, file).replace(".dat", ""))

print("Найдено файлов:", len(record_paths))

Найдено файлов: 21837


In [6]:
db_csv = "/kaggle/input/ptb-xl-dataset/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/ptbxl_database.csv"
meta = pd.read_csv(db_csv)

meta["record_name"] = meta["filename_hr"].apply(lambda x: os.path.basename(x))
meta["record_name"] = meta["record_name"].str.replace(".dat", "")

In [ ]:
fs = 500  # частота дискретизации
records = []
for path in tqdm(record_paths, desc="Обработка сигналов"):
    record = wfdb.rdrecord(path)
    signal = record.p_signal[:, 1]  # возьмём отведение II
    record_name = record.record_name

    try:
        signals, info = nk.ecg_process(signal, sampling_rate=fs)
        cleaned_signal = signals['ECG_Clean']

        q_peaks_idx = np.where(signals['ECG_Q_Peaks'] == 1)[0]
        r_peaks_idx = np.where(signals['ECG_R_Peaks'] == 1)[0]
        t_peaks_idx = np.where(signals['ECG_T_Peaks'] == 1)[0]
        s_peaks_idx = np.where(signals['ECG_S_Peaks'] == 1)[0]
        p_peaks_idx = np.where(signals['ECG_P_Peaks'] == 1)[0]

        q_peaks = np.mean(cleaned_signal[q_peaks_idx]) if len(q_peaks_idx) > 0 else 0
        r_peaks = np.mean(cleaned_signal[r_peaks_idx]) if len(r_peaks_idx) > 0 else 0
        s_peaks = np.mean(cleaned_signal[s_peaks_idx]) if len(s_peaks_idx) > 0 else 0
        p_peaks = np.mean(cleaned_signal[p_peaks_idx]) if len(p_peaks_idx) > 0 else 0
        t_peaks = np.mean(cleaned_signal[t_peaks_idx]) if len(t_peaks_idx) > 0 else 0

        mean_signal = np.mean(cleaned_signal)
        std_signal = np.std(cleaned_signal)

        heart_rate = np.mean(signals['ECG_Rate'])

        records.append({
            "record_name": record_name,
            "mean_signal": mean_signal, 
            "std_signal": std_signal, 
            "Q_peaks": q_peaks,
            "R_peaks": r_peaks,
            "T_peaks": t_peaks,
            "P_peaks": p_peaks,
            "S_peaks": s_peaks,
            "heart_rate": heart_rate
        })
    except Exception as e:
        print(f'{e}')
        continue

features_df = pd.DataFrame(records)
print("Извлечено признаков:", features_df.shape)

Обработка сигналов:   0%|          | 36/21837 [00:05<55:10,  6.59it/s] /usr/local/lib/python3.11/dist-packages/neurokit2/signal/signal_phase.py:95: RuntimeWarning: invalid value encountered in remainder
  prophase = np.mod(np.angle(scipy.signal.hilbert(signal)), pi2)
Обработка сигналов:   1%|          | 175/21837 [00:26<53:16,  6.78it/s]  /usr/local/lib/python3.11/dist-packages/neurokit2/signal/signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
Обработка сигналов:   1%|          | 177/21837 [00:27<42:53,  8.42it/s]

cannot convert float NaN to integer


Обработка сигналов:   1%|          | 261/21837 [00:40<55:02,  6.53it/s]/usr/local/lib/python3.11/dist-packages/neurokit2/signal/signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
Обработка сигналов:   1%|          | 263/21837 [00:40<42:01,  8.56it/s]

integer division or modulo by zero


Обработка сигналов:   2%|▏         | 448/21837 [01:08<51:31,  6.92it/s]  /usr/local/lib/python3.11/dist-packages/neurokit2/signal/signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
Обработка сигналов:   2%|▏         | 450/21837 [01:08<41:52,  8.51it/s]

...

Обработка сигналов:  31%|███       | 6803/21837 [17:25<41:31,  6.03it/s]/usr/local/lib/python3.11/dist-packages/neurokit2/signal/signal_phase.py:95: RuntimeWarning: invalid value encountered in remainder
  prophase = np.mod(np.angle(scipy.signal.hilbert(signal)), pi2)
Обработка сигналов:  31%|███▏      | 6829/21837 [17:29<39:32,  6.33it/s]/usr/local/lib/python3.11/dist-packages/neurokit2/signal/signal_phase.py:95: RuntimeWarning: invalid value encountered in remainder
  prophase = np.mod(np.angle(scipy.signal.hilbert(signal)), pi2)
Обработка сигналов:  32%|███▏      | 6886/21837 [17:38<38:56,  6.40it/s]/usr/local/lib/python3.11/dist-packages/neurokit2/signal/signal_phase.py:95: RuntimeWarning: invalid value encountered in remainder
  prophase = np.mod(np.angle(scipy.signal.hilbert(signal)), pi2)
Обработка сигналов:  32%|███▏      | 7037/21837 [18:02<38:34,  6.40it/s]/usr/local/lib/python3.11/dist-packages/neurokit2/signal/signal_phase.py:95: RuntimeWarning: invalid value encountered in

cannot convert float NaN to integer


Обработка сигналов:  33%|███▎      | 7269/21837 [18:38<39:52,  6.09it/s]/usr/local/lib/python3.11/dist-packages/neurokit2/signal/signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
Обработка сигналов:  33%|███▎      | 7271/21837 [18:38<31:33,  7.69it/s]

cannot convert float NaN to integer


Обработка сигналов:  34%|███▎      | 7356/21837 [18:51<39:04,  6.18it/s]/usr/local/lib/python3.11/dist-packages/neurokit2/signal/signal_phase.py:95: RuntimeWarning: invalid value encountered in remainder
  prophase = np.mod(np.angle(scipy.signal.hilbert(signal)), pi2)
Обработка сигналов:  34%|███▍      | 7383/21837 [18:55<36:44,  6.56it/s]/usr/local/lib/python3.11/dist-packages/neurokit2/signal/signal_phase.py:95: RuntimeWarning: invalid value encountered in remainder
  prophase = np.mod(np.angle(scipy.signal.hilbert(signal)), pi2)
Обработка сигналов:  34%|███▍      | 7416/21837 [19:00<36:25,  6.60it/s]/usr/local/lib/python3.11/dist-packages/neurokit2/signal/signal_phase.py:95: RuntimeWarning: invalid value encountered in remainder
  prophase = np.mod(np.angle(scipy.signal.hilbert(signal)), pi2)
Обработка сигналов:  35%|███▌      | 7643/21837 [19:36<36:50,  6.42it/s]/usr/local/lib/python3.11/dist-packages/neurokit2/signal/signal_phase.py:95: RuntimeWarning: invalid value encountered in

In [ ]:
df = pd.merge(features_df, meta, on="record_name", how="inner")

df["scp_codes"] = df["scp_codes"].apply(eval)
df["labels"] = df["scp_codes"].apply(lambda d: list(d.keys()))

In [ ]:
from collections import Counter

all_labels = [lbl for lst in df["labels"] for lbl in lst]
label_counts = Counter(all_labels)

valid_labels = {lbl for lbl, count in label_counts.items() if count >= 500}
df["labels"] = df["labels"].apply(lambda lst: [x for x in lst if x in valid_labels])
df = df[df["labels"].map(len) > 0]
print("Осталось меток:", len(valid_labels))

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(df[["mean_signal", "std_signal", "Q_peaks", "R_peaks", "T_peaks", "P_peaks", "S_peaks", "heart_rate"]])

In [ ]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df["labels"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred, target_names=mlb.classes_))

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score,
hamming_loss, accuracy_score, roc_curve, auc

hamming = hamming_loss(y_test, y_pred)
subset_acc = accuracy_score(y_test, y_pred)

print(f"Hamming loss: {hamming:.3f}")
print(f"Subset accuracy: {subset_acc:.3f}")

In [ ]:
from sklearn.metrics import RocCurveDisplay

y_score = model.predict_proba(X_test)
y_score = np.array([prob[:, 1] for prob in y_score_list]).T

roc_auc = roc_auc_score(y_test, y_score, average="macro")
print(f"ROC-AUC (macro): {roc_auc:.3f}")

plt.figure(figsize=(8, 6))
for i, label in enumerate(mlb.classes_):
    fpr, tpr, _ = roc_curve(y_test[:, i], y_score[:, i])
    plt.plot(fpr, tpr, lw=2, label=f"{label} (AUC={auc(fpr, tpr):.2f})")

plt.plot([0, 1], [0, 1], 'k--', lw=1)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC-кривые")
plt.legend()
plt.show()

In [ ]:
from joblib import dump, load

dump(model, "rf_model.joblib")

In [ ]:
!zip -r hazards_results.zip /kaggle/working
from IPython.display import FileLink
FileLink(r'/kaggle/working/rf_model.joblib')